In [1]:
%load_ext autoreload
%autoreload 2
# Add parent directory into system path
import sys, os
sys.path.insert(1, os.path.abspath(os.path.normpath('..')))

import torch
from torch import nn
from torch.nn.init import calculate_gain
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f'CUDA {i}: {torch.cuda.get_device_name(i)}')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
#device = torch.device('cpu')

from utils.operator import gradient
from models import M4, MLP_PINN

torch.set_default_dtype(torch.float32)
torch.set_printoptions(threshold=10_000)
torch.set_printoptions(linewidth=200)
torch.set_printoptions(edgeitems=3)

import os
from utils.dataset_generator import ImplicitDataset, TestDataset, batch_loader

dataset_name = '../datasets/box_1f0_gyroid_4pi'
output_stl = dataset_name+'.stl'
train_dataset = ImplicitDataset.from_file(file=dataset_name+'_train.npz', device=device)
test_dataset = TestDataset(dataset_name+'_test.npz', device=device)
print(train_dataset)
print(test_dataset)

CUDA 0: NVIDIA GeForce GTX 1650 Ti
ImplicitDataset (64000 points)
TestDataset (1000000 points, 1000000 points, 1000000 points)


In [2]:
def loss_SDF(model, x, sdfs):
    y = model(x)
    return torch.mean((y - sdfs)**2)

def loss_PDE(model, x):
    x.requires_grad_(True)
    y = model(x)
    norm_grad = gradient(y, x).norm(dim=1)
    return torch.mean((norm_grad - 1)**2)

def loss_grad(model, x, grad):
    x.requires_grad_(True)
    y = model(x)
    p = gradient(y, x)
    return torch.mean((p - grad).norm(dim=1))

def loss_cosine_similarity(model, x, grad):
    x.requires_grad_(True)
    y = model(x)
    p = gradient(y, x)
    norm_p = torch.linalg.norm(p, dim=1)
    norm_g = torch.linalg.norm(grad, dim=1)
    return torch.mean(-torch.einsum('ij,ij->i', p, grad)/norm_p/norm_g)

def loss_second_order_gradient(model, x):
    x.requires_grad_(True)
    y = model(x)
    grad = gradient(y, x)
    norm_grad = grad.norm(dim=1)
    grad_grad = gradient(grad, x)
    return torch.mean(grad_grad ** 2)

def loss_relu_residual(model, x):
    x.requires_grad_(True)
    y = model(x)
    p = gradient(y, x)
    norm_p = torch.linalg.norm(p, dim=1)
    return torch.nn.ReLU()(norm_p - 1).mean()

In [5]:
# Test PINN
import numpy as np
from utils.jacobian import extend, JacobianMode

_points = train_dataset.points.clone()
_sdfs = train_dataset.sdfs.clone()
_grads = train_dataset.grads.clone()

net = MLP_PINN(N_layers=8, width=32, activation=nn.Softplus(30), last_activation=nn.Softplus(30)).to(device)

extend(net, (3,))

batch_sizes = [1000]
K_eigvals = [[], [], [], [], [], []]
    
for bs in batch_sizes:
    print(f'Batch size: {bs}')
    for points, sdfs, grads in batch_loader(_points, _sdfs, _grads, batch_size=bs):
        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_PDE(net, points)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[1].append(torch.linalg.eigvals(K).abs().sum())

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_grad(net, points, grads)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[2].append(torch.linalg.eigvals(K).abs().sum())

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_cosine_similarity(net, points, grads)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[3].append(torch.linalg.eigvals(K).abs().sum())

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_second_order_gradient(net, points)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[4].append(torch.linalg.eigvals(K).abs().sum())

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_relu_residual(net, points)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[5].append(torch.linalg.eigvals(K).abs().sum())

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_SDF(net, points, sdfs)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[0].append(torch.linalg.eigvals(K).abs().sum())

        break

    for i,vals in enumerate(K_eigvals):
        print(f'{i}: {torch.mean(torch.tensor(vals))}')

    print('-'*10)

Batch size: 1000
0: 0.00014489727618638426
1: 1282.265625
2: 1282.2659912109375
3: 1282.7261962890625
4: 1282.265869140625
5: 1282.2657470703125
----------


In [7]:
# Test M4
import numpy as np
from models import M4

_points = train_dataset.points.clone()
_sdfs = train_dataset.sdfs.clone()
_grads = train_dataset.grads.clone()

net = M4(N_layers=8, width=32, activation=nn.Softplus(30), last_activation=nn.Softplus(30)).to(device)
# summary(
#     net, (1,3), depth=3, verbose=2, col_width=16,
#     row_settings=["var_names"], 
#     col_names=["kernel_size", "input_size", "output_size", "num_params"]
# )

extend(net, (3,))

batch_sizes = [1000]
K_eigvals = [[], [], [], [], [], []]
    
for bs in batch_sizes:
    print(f'Batch size: {bs}')
    for points, sdfs, grads in batch_loader(_points, _sdfs, _grads, batch_size=bs):

        

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_PDE(net, points)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[1].append(torch.linalg.eigvals(K).abs().sum())

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_grad(net, points, grads)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[2].append(torch.linalg.eigvals(K).abs().sum())

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_cosine_similarity(net, points, grads)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[3].append(torch.linalg.eigvals(K).abs().sum())

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_second_order_gradient(net, points)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[4].append(torch.linalg.eigvals(K).abs().sum())

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_relu_residual(net, points)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[5].append(torch.linalg.eigvals(K).abs().sum())

        net.zero_grad()
        with JacobianMode(net):
            _loss = loss_SDF(net, points, sdfs)
            _loss.backward()
            jac = net.jacobian()
            K = jac @ jac.T
            K_eigvals[0].append(torch.linalg.eigvals(K).abs().sum())

    for i,vals in enumerate(K_eigvals):
        print(f'{i}: {torch.mean(torch.tensor(vals))}')

    print('-'*10)

Batch size: 1000
0: 0.001178080914542079
1: 0.001699618762359023
2: 0.0006427342887036502
3: 0.03387424722313881
4: 0.002335398457944393
5: 0.0
----------


In [10]:
zero_lambda = torch.mean(torch.tensor(K_eigvals[0]))
for i,vals in enumerate(K_eigvals):
        print(f'{i}: {zero_lambda/(torch.mean(torch.tensor(vals))+1e-9)}')

0: 0.9999991059303284
1: 0.6931437849998474
2: 1.8329178094863892
3: 0.03477806970477104
4: 0.5044451355934143
5: 1178081.0
